In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

data = pd.read_csv("used_car_train_20200313_revised.csv")
data_test = pd.read_csv("used_car_testA_20200313_revised.csv")

# data.count()

In [2]:
X = data.dropna(axis=0)
y = X['price']
X = X.drop('price',axis=1) #dropped about 15000 rows with missing values
# X.count()

In [3]:
def preprocess(X_train):# transform 'notRepairedDamage column from 0.0, -, 1.0 to 0,1,2 as int'
    new_col = X_train.notRepairedDamage.map(lambda x: 1 if x == '-' else int(float(x))*2)
    X_train = X_train.drop('notRepairedDamage',axis=1)
    X_train = X_train.join(new_col)
    return X_train

(X_train, X_valid, y_train, y_valid) = train_test_split(X, y, test_size=0.1)
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)


### Decision Tree Regressor

In [4]:
%%time
from sklearn.tree import DecisionTreeRegressor
import timeit

dt_model = DecisionTreeRegressor(random_state=0)
dt_model.fit(X_train, y_train)
preds = dt_model.predict(X_valid)
dt_mae = mean_absolute_error(preds, y_valid)
print("mae for Decision Tree model is {}".format(dt_mae))

mae for Decision Tree model is 889.9897711384208
CPU times: user 13 s, sys: 164 ms, total: 13.2 s
Wall time: 21.6 s


In [8]:
import sklearn.tree as tree
tree.plot_tree(dt_model)

Error in callback <function flush_figures at 0x10eec6d40> (for post_execute):


KeyboardInterrupt: 

### Random Forest

In [ ]:
%%time
# num_estimators = [5, 10, 20]
# for n in num_estimators:   
X_train_seg = X_train.iloc[0:80000]
y_train_seg = y_train[X_train_seg.index]
rf_model = RandomForestRegressor(n_estimators=1, criterion='mae',verbose=True, random_state=0)
rf_model.fit(X_train_seg, y_train_seg)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:

preds = rf_model.predict(X_valid)
mae = mean_absolute_error(preds, y_valid)
print("n_estimators {}, MAE is {}".format(10, mae))